## Comparing models for our different tasks

In this Notebook, we are going to use another model, Flan-T5-large in parallel to Mistral-7B and see how it behaves.

Flan-T5-Large is indeed smaller, will run without GPU and use only 4 GB of RAM, but is it up to the task?

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI, HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain pipeline

We are now going to define two different LLM endpoints, and two different Langchain pipelines.

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLM definition
llm = VLLMOpenAI(           # we are using the vLLM OpenAI-compatible API client. But the Model is running on OpenShift, not OpenAI.
    openai_api_key="EMPTY",   # and that is why we don't need and OpenAI key for this.
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Flan-T5-Small LLM Inference Server URL
inference_server_url_flan_t5 = "http://llm-flant5.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm_flant5 = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flan_t5,
    max_new_tokens=96,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

The **template** will be the same for both models.

In [ ]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text, then ask a question about it. Give a precise and as concise as possible answer to this question.

### TEXT:
{text}

### QUESTION:
{query}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

And we can now create two **conversation** objects that we will use to query the two models.

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
conversation_flant5 = LLMChain(llm=llm_flant5,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the models!

In this example, we are only going to query one claim and see what happens. Of course, feel free to try with different ones.

In [ ]:
filename = 'claims/claim1.json'

# Opening JSON file
claims = {}
with open(filename, 'r') as file:
    data = json.load(file)
claims[filename] = data

# Analyze the claim
print(f"***************************")
print(f"* Claim: {filename}")
print(f"***************************")
print("Original content:")
print("-----------------")
print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
print('Analysis with Mistral-7B:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")
print('Analysis with Flan-T5-Large:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation_flant5.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation_flant5.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation_flant5.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")

As you can see, Flan-T5-Large is faster to produce some of the results as it's a 770 Million parameters model only. However those results are less accurate or detailed. So it works to some extent, but the results are nowhere near the ones from Mistral-7B, which is a 7 Billion parameter.

The art of working with LLM is to find the right balance between the performance and accuracy you require, and the resources it takes along with the involved costs.

Therefore it's important to have confidence checks in place to make sure that as your data changes, or your model evolves, you always get the behaviour you expected.